In [ ]:
import pandas as pd

# Load your dataset
data = pd.read_excel('21-23_dataset.xlsx')

# Convert date to datetime
data['date_committed'] = pd.to_datetime(data['date_committed'])

# Create additional features
data['day_of_week'] = data['date_committed'].dt.dayofweek
data['month'] = data['date_committed'].dt.month
data['hour'] = pd.to_datetime(data['time_committed']).dt.hour

# Group by date to create daily features
daily_data = data.groupby('date_committed').agg({'next_day_crime': 'max'}).reset_index()

# Create lag features and rolling averages
daily_data['lag_crime'] = daily_data['next_day_crime'].shift(1)
daily_data['rolling_mean'] = daily_data['next_day_crime'].rolling(window=7).mean()
daily_data['day_of_week'] = data['day_of_week']
daily_data['month'] = data['month']
daily_data['hour'] = data['hour']

<ipython-input-42-f968869e5673>:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['hour'] = pd.to_datetime(data['time_committed']).dt.hour


In [ ]:
data.head()

,barangay,incident_type,date_committed,time_committed,next_day_crime,day_of_week,month,hour
0,BALONG-BATO,ACTS OF LASCIVIOUSNESS - RPC Art. 336,2021-01-01,08:30,0,4,1,8
1,NaN,NaN,2021-01-02,00:00,0,5,1,0
2,NaN,NaN,2021-01-03,00:00,1,6,1,0
3,SANTA LUCIA,ANTI-VIOLENCE AGAINST WOMEN AND THEIR CHILDREN...,2021-01-04,01:00,0,0,1,1
4,BALONG-BATO,THEFT - RPC Art. 308,2021-01-04,05:50,0,0,1,5


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Drop NaN values created by lag and rolling features
daily_data.dropna(inplace=True)

# Define features and target
X = daily_data[['day_of_week', 'month', 'hour', 'lag_crime', 'rolling_mean']]
y = daily_data['next_day_crime']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train Random Forest Classifier
rf_model = RandomForestClassifier() #n_estimators=100, random_state=42
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.62      0.57      0.59        95
           1       0.68      0.73      0.70       121

    accuracy                           0.66       216
   macro avg       0.65      0.65      0.65       216
weighted avg       0.66      0.66      0.66       216



In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Prepare time series data
time_series_data = daily_data.set_index('date_committed')['next_day_crime']

# Fit ARIMA model
arima_model = ARIMA(time_series_data, order=(1, 0, 1))  # Adjust order based on your data
arima_result = arima_model.fit()

# Forecasting
forecast = arima_result.forecast(steps=30)  # Forecast for the next 7 days
print(forecast)

# Plotting the forecast
#plt.figure(figsize=(12, 6))
#plt.plot(time_series_data, label='Historical Crime Data')
#plt.plot(forecast, label='Forecasted Crime Data', color='red')
#plt.legend()
#plt.show()


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


1078    0.573263
1079    0.528421
1080    0.553531
1081    0.539470
1082    0.547344
1083    0.542935
1084    0.545404
1085    0.544021
1086    0.544795
1087    0.544362
1088    0.544605
1089    0.544469
1090    0.544545
1091    0.544502
1092    0.544526
1093    0.544513
1094    0.544520
1095    0.544516
1096    0.544518
1097    0.544517
1098    0.544518
1099    0.544517
1100    0.544518
1101    0.544517
1102    0.544518
1103    0.544518
1104    0.544518
1105    0.544518
1106    0.544518
1107    0.544518
Name: predicted_mean, dtype: float64


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
